In [6]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [7]:
import folium
import pandas as pd

In [8]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [12]:
from js import fetch
import io

# Cargar el dataset desde URL
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Ver los primeros registros (opcional)
spacex_df.head()

# Obtener los sitios de lanzamiento únicos con sus coordenadas
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)

# Crear un mapa base centrado en EE.UU.
launch_map = folium.Map(location=[28.5, -80.5], zoom_start=5)

# Agregar marcadores para cada sitio de lanzamiento
for index, site in launch_sites_df.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(launch_map)

# Mostrar el mapa interactivo
launch_map

In [13]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [14]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [23]:
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyter-leaflet

<class 'SyntaxError'>: invalid syntax (<ipython-input-23-d21f0175124a>, line 1)

In [25]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [32]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Obtener los sitios de lanzamiento únicos con coordenadas
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)

# Agregar círculos y marcadores para cada sitio
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Agregar el círculo con popup
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='#000000',
        fill=True,
        fill_opacity=0.4
    ).add_child(folium.Popup(site_name)).add_to(site_map)
    
    # Agregar el marcador con etiqueta de texto
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % site_name,
        )
    ).add_to(site_map)

# Mostrar el mapa
site_map

In [37]:
import folium
from folium import plugins
from folium.plugins import MarkerCluster

# Coordenadas base: NASA Johnson Space Center
nasa_coordinate = [29.559684, -95.083097]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Agrupador para que los marcadores no saturen
marker_cluster = MarkerCluster().add_to(site_map)

# Iterar sobre el DataFrame y agregar marcador por cada lanzamiento
for index, row in spacex_df.iterrows():
    # Extraer coordenadas, nombre de sitio, y estado (éxito/fallo)
    lat = row['Lat']
    lon = row['Long']
    site = row['Launch Site']
    outcome = row['class']
    
    # Color del marcador según éxito/fallo
    color = 'green' if outcome == 1 else 'red'
    status = 'Success' if outcome == 1 else 'Failure'
    
    # Añadir marcador con ícono de color
    folium.Marker(
        location=[lat, lon],
        popup=f"{site} - {status}",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(marker_cluster)

# Mostrar mapa
site_map

In [38]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [39]:
marker_cluster = MarkerCluster()

In [40]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [50]:
# Crear MarkerCluster
marker_cluster = MarkerCluster()

# Por cada fila del DataFrame crear el marker y agregarlo al cluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color']),
        popup=f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

# Agregar el cluster al mapa
site_map.add_child(marker_cluster)

# Mostrar mapa
site_map

In [44]:
from folium.plugins import MousePosition

In [49]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [48]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [53]:
from folium.features import DivIcon
import folium

# Coordenadas ejemplo (reemplaza con las coordenadas reales que marques con MousePosition)
launch_site_lat, launch_site_lon = 28.573255, -80.646895  # Ejemplo: KSC LC-39A
coastline_lat, coastline_lon = 28.56367, -80.57163         # Punto cercano en la costa

# Calcular distancia en km usando tu función calculate_distance (de la tarea anterior)
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Crear marcador para el punto de la costa con distancia en el icono
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 14px; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    ),
    popup="Coastline Point"
)

# Agregar marcador al mapa
distance_marker.add_to(site_map)

# Crear PolyLine entre sitio de lanzamiento y punto de costa
lines = folium.PolyLine(
    locations=[ [launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon] ],
    weight=3,
    color='blue',
    opacity=0.7
)

# Agregar PolyLine al mapa
site_map.add_child(lines)

# También puedes agregar un marcador para el sitio de lanzamiento para referencia visual
launch_marker = folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color='green', icon='rocket')
)
launch_marker.add_to(site_map)

# Mostrar mapa actualizado
site_map


In [56]:
import folium
from folium import DivIcon
import math

# Función para calcular distancia entre dos puntos (Haversine)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) ** 2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Coordenadas del sitio de lanzamiento (KSC LC-39A)
launch_site = [28.5954, -80.65322]

# Coordenadas de puntos cercanos
coast = [28.56367, -80.57163]
city = [28.6128, -80.8076]           # Titusville
rail = [28.5721, -80.6678]           # Estimado
highway = [28.57205, -80.6497]       # Estimado

# Calcular distancias
dist_coast = calculate_distance(*launch_site, *coast)
dist_city = calculate_distance(*launch_site, *city)
dist_rail = calculate_distance(*launch_site, *rail)
dist_highway = calculate_distance(*launch_site, *highway)

# Crear mapa
site_map = folium.Map(location=launch_site, zoom_start=12)

# Agregar marcador de sitio de lanzamiento
folium.Marker(launch_site, popup='Launch Site', icon=folium.Icon(color='green')).add_to(site_map)

# Lista de puntos y distancias
puntos = {
    "Coastline": (coast, dist_coast, 'blue'),
    "City (Titusville)": (city, dist_city, 'red'),
    "Railway": (rail, dist_rail, 'purple'),
    "Highway": (highway, dist_highway, 'orange')
}

# Añadir marcadores, distancias y líneas
for label, (coord, dist, color) in puntos.items():
    folium.Marker(coord, popup=label, icon=folium.Icon(color=color)).add_to(site_map)

    folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 12px; color:{};"><b>{:.2f} KM</b></div>'.format(color, dist),
        )
    ).add_to(site_map)

    folium.PolyLine(locations=[launch_site, coord], color=color, weight=2).add_to(site_map)

# Mostrar mapa
site_map